# Máquinas Térmicas
 _Prof. Tito Dias Júnior_

### Ciclo de Rankine Simples Real


### **Exemplo:**
Uma usina de potência a vapor de água opera segundo o ciclo mostrado abaixo.
Se a eficiência isentrópica da turbina é de 87% e a eficiência isentrópica da bomba é
de 85%, determine (a) a eficiência térmica do ciclo e (b) a potência líquida da usina
para um fluxo de massa de 15 kg/s.



![Esquema](Exemplo_10.2.png "Esquema do Ciclo de Potência")

## Solução

Para a solução do problema devemos analisar o diagrama T-s para o ciclo ideal de Rankine e, partindo da definição, calcular a eficiência térmica do ciclo. 

Nesse exemplo, a turbina e o compressores são reais ou seja não são perfeitamente isentrópicos, havendo aumento de entropia nos processos correspondentes. Esse aumento real de entropia pode ser medido em laboratório e utilizado para calccular a efeciência isentrópica de cada equipamento.

Além disso, no esquema da figura anterior, observamos queda de pressão entre a entrada e a saída da caldeira em razão da perda de carga, ou dissipação de energia devido ao escoamento. O mesmo ocorre no condensador.

Podemos construir o diagrama abaixo, considerando o ciclo em regime permanente e as perdas de carga no evaporador e condensador, considerando as esficiências isentrópicas da bomba e da turbina isentrópicas.

![Esquema](Diagrama_10.2.png "Diagrama T-s do Ciclo de Potência")

Lembrando que a eficiência térmica é a relação entre trabalho líquido e o calor fornecido na caldeira, para sua determinação devemos calcular, além do calor fornecido ($Q_H$), o trabalho produzido pela turbina ($W_{turb}$) e o trabalho consumido pela bomba ($W_{bomb}$).

O trabalho consumido pela bomba pode ser calculado de duas formas. Uma seria determinando a entalpia do estado 2 usando sua eficiência isentrópica e o estado 1 ($T_1=38°C$ e $P_1=9kPa$), e em seguida o trabalho consumido pela bomba. A outra seria considerar o o volume do líquido comprimido aproximadamente constante e calcular o trabalho de compressão como função somente da variação de pressão, ou seja $W_{bomb, real}=\frac{v_1(P_2-P_1)}{\eta_{bomb}}$.




Começamos obtendo as propriedades do fluido de trabalho (Água) nos estados de interesse. Para isso vamos usar a biblioteca de propriedades de fluidos CoolProp (http://www.coolprop.org) e definir a água (water) como fluido de trabalho.

In [3]:
from CoolProp import CoolProp as cp
fluido = 'Water'

Para o estado 1, temos líquido a $P_1=5,62kPa = 0,009MPa$ e $T_1=35°C$:


In [4]:
P1 = 0.00568 #Unidade em MPa, a conversão para Pa será feita na chamada à função abaixo.
T1 = 35 + 273.15    # Temperatura em K
h1 = cp.PropsSI('H', 'P', P1 * 1E6, 'T', T1, fluido) * 1E-3 #Determinando a entalpia e convertendo para kJ/kg
v1 = 1/cp.PropsSI('D', 'P', P1 * 1E6, 'T', T1, fluido)  #Determinando a densidade e invertendo para obter o volume específico (m^3/kg)
s1 = cp.PropsSI('S', 'P', P1 * 1E6, 'T', T1, fluido) * 1E-3 #Determinando a entropia e convertendo para kJ/kg
Q1 = cp.PropsSI('Q', 'P', P1 * 1E6, 'T', T1, fluido)        #Determinando a temperatura em K
print('h1= {:.3f} kJ/kg'.format(h1))
print('v1= {:.6f} m^3/kg'.format(v1))
print('s1= {:.3f} kJ/kg'.format(s1))
print('T1= {:.1f} °C'.format(T1-273.15))


h1= 146.634 kJ/kg
v1= 0.001006 m^3/kg
s1= 0.505 kJ/kg
T1= 35.0 °C


Do enunciado, temos que $\eta_{bomb}=0,85$, $P_2=0,5MPa$ e podemos calcular o trabalho consumido pela bomba:

In [5]:
eta_bomb = 1 # Eficiência isentrópica da bomba
P2 = 0.5        # Pressão em MPa
W_bomb = v1*(P2-P1)/eta_bomb * 1e3 #Fator de conversão de MPa para kPa
print('W_bomb = {:.3f} kJ/kg'.format(W_bomb))

W_bomb = 0.497 kJ/kg


Para calcularmos o trabalho produzido pela turbina, utilizamos as entalpias dos estados 5 e 6s, equivalente a uma turbina isentrópica e multiplicamos pela eficiência isentrópica da turbina ($\eta_{turb}=0,87$):

In [6]:
eta_turb = 1 # Eficiência isentrópica da turbina
# Estado 5
P5 = 0.5
T5 = 300 + 273.15
h5 = cp.PropsSI('H', 'P', P5 * 1E6, 'T', T5, fluido) * 1E-3
s5 = cp.PropsSI('S', 'P', P5 * 1E6, 'T', T5, fluido) * 1E-3
print(h5)
# Estado 6
P6 = 0.0568 # P6 = 10kPa
s6s = s5    # Estado 6s de uma turbina isentrópica
h6s = cp.PropsSI('H', 'P', P6 * 1E6, 'S', s6s *1E3, fluido) * 1E-3
Q6s = cp.PropsSI('Q', 'P', P6 * 1E6, 'S', s6s *1E3, fluido) 
print(h6s)
print('Q6s = ', Q6s)

# Trabalho na turbina real = trabalho na turbina isentrópica vezes a efeciência isentrópica
W_turb = eta_turb*(h5-h6s)
print('W_turb = {:.3f} kJ/kg'.format(W_turb))

3064.5536433601965
2618.9619322838175
Q6s =  0.9862476649180588
W_turb = 445.592 kJ/kg


Agora nos resta calcular o calor fornecido na caldeira $Q_H=(h_4-h_3)$ :

In [7]:
# Estado 3:
P3 = 0.5
T3 = 35 + 273.15
h3 = cp.PropsSI('H', 'P', P3 * 1E6, 'T', T3, fluido) * 1E-3
print(h3)

# Estado 4:
P4 = 0.5
T4 = 300 + 273.15
h4 = cp.PropsSI('H', 'P', P4 * 1E6, 'T', T4, fluido) * 1E-3
print(h4)

# Calor fornecido pela caldeira
Q_H = h4-h3
print('Q_H  = {:.3f} kJ/kg'.format(Q_H))

147.0781423846125
3064.5536433601965
Q_H  = 2917.476 kJ/kg


Assim a eficiência pode ser calculada $\eta = \frac{W_turb-W_{bomb}}{Q_H}$:

In [8]:
# Cálculo da eficiência
eta = (W_turb-W_bomb)/Q_H
print('Eficiência = {:.3f}'.format(eta))

Eficiência = 0.153


b) Para determinar a potência produzida pelo ciclo para a vazão mássica de fluido de trabalho prescrita:

In [9]:
mQH = 20E6/3600 # 20 MJ/h
print(mQH)
m = mQH/(Q_H*1E3)
mW_turb = W_turb*m
mW_bomb = W_bomb*m
print('m = {:.6f} kg/s'.format(m))
print('Potência Turbina = {:.3f} W'.format(mW_turb*1E3))
print('Potência Bomba = {:.3f} W'.format(mW_bomb*1E3))

5555.555555555556
m = 0.001904 kg/s
Potência Turbina = 848.511 W
Potência Bomba = 0.947 W


In [25]:
print('=========SUMÁRIO============')
print('============================')
print('ebomb = {:.2f} <> eturb = {:.2f}'.format(eta_bomb,eta_turb))
print('----------------------------')
print('W_bomb     = {:.3f} kJ/kg'.format(W_bomb))
print('W_turb     = {:.3f} kJ/kg'.format(W_turb))
print('Q_H        = {:.3f} kJ/kg'.format(Q_H))
print('Eficiência = {:.3f}'.format(eta))
print('Potência   = {:.3f} kW'.format(mW_turb))
print('============================')


=========SUMÁRIO============
ebomb = 1.00 <> eturb = 1.00
----------------------------
W_bomb     = 0.497 kJ/kg
W_turb     = 445.592 kJ/kg
Q_H        = 2917.476 kJ/kg
Eficiência = 0.153
Potência   = 0.849 kW
